# PC Specific ScanImage --> NWB (SCRIPT)

## create NWB file

### set directories

In [1]:
import os
import glob
import re

# dataPath = "/home/pac/Documents/Python/nwb/scanimage/rawData"
# outputNWBpath = f"/home/pac/Documents/Python/nwb/scanimage/nwbOutput/{experimentID}.nwb"

experimentID = 'AA0315'

dataPath = "/media/DATA/backups/sutter2P_backup/D_drive/dataSince20210304"
outputNWBpath = os.path.join(dataPath,experimentID,f"{experimentID}.nwb")
motionCorrectedTifDir = 'NoRMCorred'

experimentDir = os.path.join(dataPath,experimentID)
experiment_mat = f"{experimentID}_anmlROI_stimTable.mat"
moCorrMat = f"{experimentID}_NoRMCorreParams.mat"
fluorescenceMat = f"{experimentID}_tifFileList.mat"


pupilMat = f"{experimentID}_pulsePupilUVlegend2P_s.mat"
if (os.path.exists(os.path.join(experimentDir,pupilMat)) or 
    os.path.exists(os.path.join(experimentDir,pupilMat.replace('_s.mat','.mat')))):
    print(f"pupil data exists for {experimentID}")
    addPupillometry = True



pupil data exists for AA0315


### extract .tif image data and metadata

In [2]:
# get tif file list
import sys
sys.path.insert(1, '../')
import lib.mat2py

# get tif creation date, end write time, and frame counts
tifFileList = lib.mat2py.getMatCellArrayOfStr(os.path.join(experimentDir,experiment_mat),varPath = ['tifFileList','stim','name'])
fileTimeWrite = lib.mat2py.getMatCellArrayOfStr(os.path.join(experimentDir,experiment_mat),varPath = ['tifFileList','stim','date'])
tifFrameCounts = lib.mat2py.getMatCellArrayOfNum(os.path.join(experimentDir,experiment_mat),varPath = ['tifFileList','stim','nFrames'])
treatment = lib.mat2py.getMatCellArrayOfStr(os.path.join(experimentDir,experiment_mat),varPath = ['tifFileList','stim','treatment'])

print(list(zip(tifFileList,fileTimeWrite,tifFrameCounts,treatment)))

[('AA0315AAAA_00010_00001.tif', '10-May-2021 17:43:28', 170.0, 'preZX1'), ('AA0315AAAA_00011_00001.tif', '10-May-2021 17:46:27', 170.0, 'preZX1'), ('AA0315AAAA_00012_00001.tif', '10-May-2021 17:48:52', 170.0, 'preZX1'), ('AA0315AAAA_00013_00001.tif', '10-May-2021 17:50:12', 170.0, 'preZX1'), ('AA0315AAAA_00014_00001.tif', '10-May-2021 17:51:45', 170.0, 'preZX1'), ('AA0315AAAA_00015_00001.tif', '10-May-2021 17:53:03', 170.0, 'preZX1'), ('AA0315AAAA_00016_00001.tif', '10-May-2021 17:54:05', 170.0, 'preZX1'), ('AA0315AAAA_00017_00001.tif', '10-May-2021 17:55:41', 170.0, 'preZX1'), ('AA0315AAAA_00018_00001.tif', '10-May-2021 17:57:24', 170.0, 'preZX1'), ('AA0315AAAA_00019_00001.tif', '10-May-2021 17:58:56', 170.0, 'preZX1'), ('AA0315AAAA_00020_00001.tif', '10-May-2021 18:00:24', 170.0, 'preZX1'), ('AA0315AAAA_00021_00001.tif', '10-May-2021 18:02:33', 170.0, 'preZX1'), ('AA0315AAAA_00022_00001.tif', '10-May-2021 18:03:37', 170.0, 'preZX1'), ('AA0315AAAA_00023_00001.tif', '10-May-2021 18:04:

In [ ]:
# ROI mat ([experimentID]_moCorrROI_all.mat) will always end in _all.mat if treatment is 'none'.
# If treatment (eg preZX1 and postZX1), there may be either one ROI mat for the whole session (ending in _all.mat) 
# or an ROI .mat for each treatment (eg. [experimentID]_moCorrROI_preZX1.mat and [experimentID]_moCorrROI_postZX1.mat)

# If treatment == 'none' (should be same for all .tif), then ROI mat will be called [experimentID]_moCorrROI_all.mat
# and keys in imaging_plane and ImageSegmentation will be called 'all'

# If multiple ROI sets / ROI change with time, must have different imaging_plane for each ROI set (via create_imaging_plane)

# TwoPhotonSeries must be associated with the corresponding imaging plane
# If multiple ROI .mat, there will be a separate ImageSegmentation (stored in plane_seg) for each,
# this will reference the corresponding imaging plane.
# ROI table regions are created for each ImageSegmentation (stored in plane_seg) and house
# ROI fluorescence data.
# Each RoiResponseSeries (eg fluorescence for a given .tif) is associated with the corresponding ROI table.
roiMatPat = f"{experimentID}_moCorrROI*.mat"
roiMats = glob.glob(os.path.join(experimentDir,roiMatPat))

if len(roiMats)==1 and os.path.basename(roiMats[0])==f"{experimentID}_moCorrROI_all.mat":
    roiSet = ["all"]
    tifROI = roiSet*len(treatment)
else:
    roiSet = [re.search(f"{experimentID}_moCorrROI_(.*).mat",roiMat).group(1) for roiMat in roiMats]
    tifROI = treatment

In [4]:
tifFileList[0]

'AA0315AAAA_00010_00001.tif'

In [6]:
lib.mat2py.getTifStimParams(os.path.join(experimentDir,tifFileList[0]))

AttributeError: 'NoneType' object has no attribute 'groupdict'

### NWB file prep

#### instantiate NWB file

In [10]:
from uuid import uuid4
from pynwb import NWBHDF5IO, NWBFile, TimeSeries
from pynwb.image import ImageSeries
from pynwb.ophys import (
    CorrectedImageStack,
    Fluorescence,
    ImageSegmentation,
    MotionCorrection,
    OpticalChannel,
    RoiResponseSeries,
    TwoPhotonSeries,
)

In [ ]:
import lib.tifExtract
import importlib
importlib.reload(lib.tifExtract)
# get metadata from first tif for session start
session_start = lib.tifExtract.getSItifTime(os.path.join(experimentDir,tifFileList[0]))
session_start

In [ ]:
# refer to notebook for session description
# standardize: experiment_animal_mouseLine --> pupilPTinContrast_AA0304_C57BL6J
# see dataMapPCetal2024.txt: /media/DATA/backups/sutter2P_backup/D_drive

nwbfile = NWBFile(
    session_description=f"pupilPTinContrast_{experimentID}_C57BL6J",
    identifier=str(uuid4()),
    session_start_time=session_start,
    experimenter=[
        "Cody, Patrick",
    ],
    lab="Tzounopoulos Lab",
    institution="University of Pittsburgh",
    experiment_description="Pure tone in low and high DRC contrast with pupillometry",
    keywords=["2P", "CGC", "pupillometry", "DRC"],
    related_publications="10.1523/JNEUROSCI.0939-23.2024",
)


#### set imaging plane

In [13]:
device = nwbfile.create_device(
    name="Microscope",
    description="Sutter moveable objective microscope (MOM) with mode-locked laser light (MaiTai HP) at 100-200 mW intensity through 40x0.8NA objective (Olympus) with X-Y galvanometric scanning",
    manufacturer="Sutter",
)
optical_channel = OpticalChannel(
    name="OpticalChannel",
    description="green channel for GCaMP",
    emission_lambda=525.0,
)
# filter FF03 525/50 (Semrock) --> 525 +/- 25 (center wavelength 525): https://www.idex-hs.com/store/product-detail/ff03_525_50_25/fl-004656

In [14]:
# if multiple ROI sets / ROI change with time, must have different imaging plane for each ROI set --> usually the case with pre/post treatment (ZX1)
imgPlane = {}
for cond in roiSet:
    imgPlane[cond] = nwbfile.create_imaging_plane(
        name=f"ImagingPlane{cond}",
        optical_channel=optical_channel,
        imaging_rate=5.0,
        description="Auditory Cortex",
        device=device,
        excitation_lambda=940.0,
        indicator="GFP",
        location="ACtx",
        grid_spacing=[145.0, 145.0],
        grid_spacing_unit="micrometers",
        origin_coords=[-2.0, 4.25, 2.0],
        origin_coords_unit="meters",
    )
# “Origin coordinates are relative to bregma. First dimension corresponds to anterior-posterior axis (larger index = more anterior). 
# Second dimension corresponds to medial-lateral axis (larger index = more rightward). 
# Third dimension corresponds to dorsal-ventral axis (larger index = more ventral).

#### set two-photon series data

In [15]:
# add to nwb 2p series
import lib.tifExtract
import importlib
importlib.reload(lib.tifExtract)

fileTimesInstantiate, nFrames, frameRates, starts = [],[],[],[]
two_p_series = []
for i,tif in enumerate(tifFileList):
    imgData,fileTimeInstantiate,frameCount,frameRate = lib.tifExtract.getSItifData(os.path.join(experimentDir,tif))
    start = lib.tifExtract.secMicroSec2sec(fileTimeInstantiate-session_start)
    two_p_ser = TwoPhotonSeries(
        name=f"TwoPhotonSeries_{i:03}",
        description="Raw 2p data",
        data=imgData,
        imaging_plane=imgPlane[tifROI[i]],
        rate=frameRate,
        starting_time=start,
        unit="normalized amplitude",
        comments=f"file: {tif}, treatment: {treatment[i]}, fileTimeInstantiate: {fileTimeInstantiate}, nFrames: {frameCount})",
    )
    two_p_series.append(two_p_ser)

    fileTimesInstantiate.append(fileTimeInstantiate)
    nFrames.append(frameCount)
    frameRates.append(frameRate)
    starts.append(start)

    nwbfile.add_acquisition(two_p_ser)

In [28]:
# test file write
from lib.nwbScanImage import writeNWB
writeNWB(outputNWBpath,nwbfile)
# view with neurosift: neurosift {outputNWBpath}


#### add motion correction data
- shifts in animalDir/NoRMCorred/[animal]_NoRMCorreParams.mat
    - concatenated together of length (nTifFiles * sum(tifFileList.stim.nFrames))
    - dim of each frame is (8,8,1,2) --> can be squeezed to (8,8,2)

In [16]:
import lib.mat2py
import importlib
importlib.reload(lib.mat2py)

shifts,moCorrParams = lib.mat2py.getMoCorrShiftParams(os.path.join(experimentDir,motionCorrectedTifDir,moCorrMat),nFrames=nFrames)

In [17]:
# add processing module to include motiuon corrected data
ophys_module = nwbfile.create_processing_module(
    name="ophys", description="optical physiology processed data"
)
motion_correction = MotionCorrection(name='Motion Corrected TwoPhotonSeries')

In [18]:
for i,(tif,shift) in enumerate(zip(tifFileList,shifts)):
    imgData = lib.tifExtract.getSItifData(os.path.join(experimentDir,
                                                       motionCorrectedTifDir,
                                                       tif.replace('.tif','_NoRMCorre.tif')),getMetadata=False)

    corrected = ImageSeries(
        name="corrected",  # this must be named "corrected"
        description=f"A motion corrected image stack for acquisition {i:03}",
        data=imgData,
        unit="na",
        format="raw",
        comments=f"corrected file: {tif}",
        rate=frameRates[i],
        starting_time=starts[i],
    )

    xy_translation = TimeSeries(
        name=f"xy_translation",
        description=f"x,y translation in pixels for acquisition {i:03}",
        data=shift,
        unit="pixels",
        rate=frameRates[i],
        starting_time=starts[i],
        control_description = (moCorrParams if i==0 else None),
        comments=('control_description: NoRMCorreParams' if i==0 else ''),
    )

    motion_correction.add_corrected_image_stack(CorrectedImageStack(
        corrected=corrected,
        original=two_p_series[i],
        xy_translation=xy_translation,
        name=f"motion_corrected_TwoPhotonSeries_{i:03}"
    ))

In [ ]:
ophys_module.add(motion_correction)

In [13]:
# test file write
from lib.nwbScanImage import writeNWB
writeNWB(outputNWBpath,nwbfile)
# view with neurosift: neurosift {outputNWBpath}


#### add ROI via planeSegmentation
- in experiment dir, ROI drawn on motion corrected data saved in [animal id]_moCorrROI_*.mat
- if no treatment file takes name [animal id]_moCorrROI_all.mat otherwise it takes form [animal id]_moCorrROI_pre[treatment].mat and [animal id]_moCorrROI_post[treatment].mat
- if more than one condition / treatment, image segmentation must be associated with a separate imaging plane

In [20]:
import re

img_seg = ImageSegmentation()
ophys_module.add(img_seg)

plane_seg = {}
# usually roiMat for each treatment
for roiMat,roiCond in zip(roiMats,roiSet):    
    roiMasks = lib.mat2py.getROImasks(roiMat)

    plane_seg[roiCond] = img_seg.create_plane_segmentation(
        name=f"PlaneSegmentation_{roiCond}",
        description=f"output from segmenting the imaging plane for {roiCond}",
            imaging_plane=imgPlane[roiCond],
            reference_images=[p for p,t in zip(two_p_series,treatment) if t==roiCond],  # optional
        )

    for roiImageMask in roiMasks:
        # add image mask to plane segmentation
        plane_seg[roiCond].add_roi(image_mask=roiImageMask)


In [ ]:
from matplotlib import pyplot as plt
plt.imshow(roiMasks[0])

#### add fluorescence traces for ROIs

In [ ]:
import lib.mat2py
import importlib
importlib.reload(lib.mat2py)

In [ ]:
# load ROI fluo data from experiment
fluoROI = lib.mat2py.getROIfluo(os.path.join(experimentDir,fluorescenceMat))
len(fluoROI)

In [25]:
# roi fluorescence responses associated with a region, each region is associated with a plane segmentation (usually one per condition)
# which has corresponding IDs for the ROI - roiResponseSeries a linked to these planesegment IDs
roi_resp_series = []
for cond in plane_seg:
    rt_region = plane_seg[cond].create_roi_table_region(
        region=plane_seg[cond].id.data, description=f"ROI for {cond}"
        )
    # only get responses in the matching treatment condition
    responses = [(i,f,s,fr) for i,(f,t,s,fr) in enumerate(zip(fluoROI,tifROI,starts,frameRates)) if t==cond]
    for i,f,s,fr in responses:
        roi_resp_series.append(RoiResponseSeries(
            name=f"RoiResponseSeries_{i:03}",
            description=f"Fluorescence responses for motion corrected ROIs for TwoPhotonSeries_{i:03}",
            data=f,
            rois=rt_region,
            unit="lumens",
            rate=fr,
            starting_time=s
            ))

In [ ]:
# one fluorescence module, RoiResponseSeries is a list
fl = Fluorescence(roi_response_series=roi_resp_series)
ophys_module.add(fl)

In [34]:
# test file write
writeNWB(outputNWBpath,nwbfile)
# view with neurosift: neurosift {outputNWBpath}

#### add sound stimulus data via DynamicTable

In [27]:
# add sound stimulus data via DynamicTable
pulseNames = lib.mat2py.getMatCellArrayOfStr(os.path.join(experimentDir,experiment_mat),['pulseLegend2P','pulseName'])
pulseSets = lib.mat2py.getMatCellArrayOfStr(os.path.join(experimentDir,experiment_mat),['pulseLegend2P','pulseSet'])
tifPulse = lib.mat2py.getMatCellArrayOfStr(os.path.join(experimentDir,experiment_mat),['pulseLegend2P','tif'])
pulseNames, pulseSets = zip(*[(pulseName,pulseSet) for t,pulseName,pulseSet in zip(tifPulse,pulseNames,pulseSets) if t in tifFileList])

stimData = {
    'fileTimeInstantiate': ('time .tif file was instantiated/created',
                            [dt.strftime('%d-%b-%Y %H:%M:%S.%f') for dt in fileTimesInstantiate]),
    'file': ('name of .tif file',tifFileList),
    'nFrames': ('number of frames in .tif file',nFrames),
    'frameRate': ('frame rate of .tif file',frameRates),
    'pulseNames': ('sound stimulation pulse name',pulseNames),
    'pulseSets': ('sound stimulation pulse set',pulseSets),
    'treatment': ('treatment',treatment)
}

In [28]:
from hdmf.common import VectorData, DynamicTable

cols = []
for col,v in stimData.items():
    cols.append(
            VectorData(
            name=col,
            description=v[0],
            data=v[1],
        )
    )
stim_table = DynamicTable(
    name='stim param table',
    description='Maps sound stim parameters to .tif files',
    columns=cols,
)

nwbfile.add_stimulus(stim_table)

In [24]:
# test file write
from lib.nwbScanImage import writeNWB
writeNWB(outputNWBpath,nwbfile)
# view with neurosift: neurosift {outputNWBpath}

if addPupillometry==False:
    print('stop here, no pupillometry')

#### add pupillometry data
- behavior/pupil tracking with timeSeries for pupilDiameter
- processing/imageSeries for pupil video

In [ ]:
import lib.mat2py
import importlib
importlib.reload(lib.mat2py)

##### add pupil radius

In [29]:
import numpy as np
from pynwb.behavior import PupilTracking

behavior_module = nwbfile.create_processing_module(
    name="behavior", description="Processed behavioral data"
)


In [47]:
# split:
pupilDataProcessed = lib.mat2py.getPupilDataProcessed(os.path.join(experimentDir,pupilMat))
pupil_radii = []


In [48]:
pupilFrameFiles,pupilRadius = zip(*[(f,r) for f,r in zip(pupilDataProcessed['pupilFrameFiles'],pupilDataProcessed['pupilRadius']) 
    if f.replace('_pupilFrames.mat','.tif') in tifFileList])

In [49]:
for i,pupilFrameFile in enumerate(pupilFrameFiles):
    pupil_radii.append(
        TimeSeries(
            name=f"pupil_radius_{i:03}",
            description=f"Pupil radius extracted from the video of the right eye for TwoPhotonSeries_{i:03}",
            data=pupilRadius[i],
            rate=float(pupilDataProcessed['frameRate']),
            starting_time=starts[i],
            unit="na",
            comments=f"pupilFrameFile: {pupilFrameFile}, associated .tif file: {tifFileList[i]}"
        )
    )


In [ ]:
pupil_tracking = PupilTracking(time_series=pupil_radii, name="PupilTracking")

behavior_module.add(pupil_tracking)

In [28]:
# test file write
writeNWB(outputNWBpath,nwbfile)
# view with neurosift: neurosift {outputNWBpath}

##### add pupil video
- without video: ~1.4 GB for AA0304
- with pupil video: ~4.6 GB for AA0304

In [51]:
pupilVideoSeries = []

for i,pupilFrameFile in enumerate(pupilFrameFiles):
    pupilVideoSeries.append(
        ImageSeries(
            name=f"pupil_video_{i:03}",
            description=f"Pupil video of the right eye for TwoPhotonSeries_{i:03}",
            data=lib.mat2py.getPupilImg(os.path.join(experimentDir,pupilFrameFile)),
            rate=float(pupilDataProcessed['frameRate']),
            starting_time=starts[i],
            unit="na",
            comments=f"pupilFrameFile: {pupilFrameFile}, associated .tif file: {tifFileList[i]}"
        )
    )

In [ ]:
behavior_module.add(pupilVideoSeries)

In [58]:
# test file write
from lib.nwbScanImage import writeNWB
writeNWB(outputNWBpath,nwbfile,overWrite=True)
# view with neurosift: neurosift {outputNWBpath}

# ScanImage --> NWB function

In [ ]:
# import lib.nwbScanImage

# import importlib
# importlib.reload(lib.nwbScanImage)

# experimentID = 'AA0314'

# dataPath = "/media/DATA/backups/sutter2P_backup/D_drive/dataSince20210304"
# outputNWBpath = os.path.join(dataPath,experimentID,f"{experimentID}.nwb")

# session_description = f"pupilPTinContrast_{experimentID}_C57BL6J"
# experiment_description = "Pure tone in low and high DRC contrast with pupillometry"
# keywords = ["2P", "CGC", "pupillometry", "DRC"]

# lib.nwbScanImage.genNWBfromScanImage_pc_concat(
#     experimentID=experimentID, 
#     dataPath=dataPath, 
#     NWBoutputPath=outputNWBpath,
#     session_description=session_description,
#     experiment_description=experiment_description,
#     keywords=keywords,
#     **lib.nwbScanImage.PARAMS_nwbFilePC,
#     **lib.nwbScanImage.PARAMS_imagingPC
# )

In [ ]:
# import os
# import lib.nwbScanImage

# import importlib
# importlib.reload(lib.nwbScanImage)

# experimentID = 'AA0308'
# # /media/DATA/backups/sutter2P_backup/D_drive/ZnT3_pupil/BIN2/AA0323
# #  /media/DATA/backups/sutter2P_backup/D_drive/dataSince20210304/AA0308
# dataPath = "/media/DATA/backups/sutter2P_backup/D_drive/dataSince20210304"
# outputNWBpath = os.path.join(dataPath,experimentID,f"{experimentID}.nwb")

# session_description = f"pupilPTinContrast_{experimentID}_C57BL6J"
# experiment_description = "Pure tone in low and high DRC contrast with pupillometry"
# keywords = ["2P", "CGC", "pupillometry", "DRC"]

# lib.nwbScanImage.genNWBfromScanImage_pc_concat(
#     experimentID=experimentID, 
#     dataPath=dataPath, 
#     NWBoutputPath=outputNWBpath,
#     session_description=session_description,
#     experiment_description=experiment_description,
#     keywords=keywords,
#     **lib.nwbScanImage.PARAMS_nwbFilePC,
#     **lib.nwbScanImage.PARAMS_imagingPC
# )

In [1]:
# after fix to file by file
import sys
sys.path.insert(1, '../')
import os
import lib.nwbScanImage

import importlib
importlib.reload(lib.nwbScanImage)

# AA0314 took ~3 minutes to run
experimentID = 'AA0314'
# /media/DATA/backups/sutter2P_backup/D_drive/ZnT3_pupil/BIN2/AA0323
#  /media/DATA/backups/sutter2P_backup/D_drive/dataSince20210304/AA0308
dataPath = "/media/DATA/backups/sutter2P_backup/D_drive/dataSince20210304"
outputNWBpath = os.path.join(dataPath,experimentID,f"{experimentID}.nwb")

session_description = f"pupilPTinContrast_{experimentID}_C57BL6J"
experiment_description = "Pure tone in low and high DRC contrast with pupillometry"
keywords = ["2P", "CGC", "pupillometry", "DRC"]

lib.nwbScanImage.genNWBfromScanImage_pc(
    experimentID=experimentID, 
    dataPath=dataPath, 
    NWBoutputPath=outputNWBpath,
    session_description=session_description,
    experiment_description=experiment_description,
    keywords=keywords,
    **lib.nwbScanImage.PARAMS_nwbFilePC,
    **lib.nwbScanImage.PARAMS_imagingPC
)

stim
[('AA0314AAAA_00010_00001.tif', 170.0, 'preZX1'), ('AA0314AAAA_00011_00001.tif', 170.0, 'preZX1'), ('AA0314AAAA_00012_00001.tif', 170.0, 'preZX1'), ('AA0314AAAA_00013_00001.tif', 170.0, 'preZX1'), ('AA0314AAAA_00014_00001.tif', 170.0, 'preZX1'), ('AA0314AAAA_00015_00001.tif', 170.0, 'preZX1'), ('AA0314AAAA_00016_00001.tif', 170.0, 'preZX1'), ('AA0314AAAA_00017_00001.tif', 170.0, 'preZX1'), ('AA0314AAAA_00018_00001.tif', 170.0, 'preZX1'), ('AA0314AAAA_00019_00001.tif', 170.0, 'preZX1'), ('AA0314AAAA_00020_00001.tif', 170.0, 'preZX1'), ('AA0314AAAA_00021_00001.tif', 170.0, 'preZX1'), ('AA0314AAAA_00022_00001.tif', 170.0, 'preZX1'), ('AA0314AAAA_00023_00001.tif', 170.0, 'preZX1'), ('AA0314AAAA_00024_00001.tif', 170.0, 'preZX1'), ('AA0314AAAA_00025_00001.tif', 170.0, 'preZX1'), ('AA0314AAAA_00026_00001.tif', 120.0, 'preZX1'), ('AA0314AAAA_00029_00001.tif', 120.0, 'preZX1'), ('AA0314AAAA_00030_00001.tif', 120.0, 'preZX1'), ('AA0314AAAA_00031_00001.tif', 120.0, 'preZX1'), ('AA0314AAAA_0

/home/pac/Documents/Python/nwb/venvNWB/lib/python3.12/site-packages/pynwb/file.py:472: UserWarning: Date is missing timezone information. Updating to local timezone.
  args_to_set['session_start_time'] = _add_missing_timezone(session_start_time)


added 2P data
added motion correction data
added ROI segmentation data


/home/pac/Documents/Python/nwb/venvNWB/lib/python3.12/site-packages/hdmf/container.py:536: UserWarning: The linked table for DynamicTableRegion 'rois' does not share an ancestor with the DynamicTableRegion.
  child._validate_on_set_parent()


added fluorescence trace data for ROIs
added stim table data
found pupillometry data for AA0314
added pupil radius data
added pupil video data
NWB write success to: /media/DATA/backups/sutter2P_backup/D_drive/dataSince20210304/AA0314/AA0314.nwb


In [ ]:
# after fix to file by file
import sys
sys.path.insert(1, '../')
import os
import lib.nwbScanImage

import importlib
importlib.reload(lib.nwbScanImage)

# AA0314 took ~3 minutes to run
# /media/DATA/backups/sutter2P_backup/D_drive/ZnT3_pupil/BIN2/AA0323
#  /media/DATA/backups/sutter2P_backup/D_drive/dataSince20210304/AA0308
dataPath = "/media/DATA/backups/sutter2P_backup/D_drive/ZnT3_pupil/BIN2"
experimentID = "AA0330"
outputNWBpath = os.path.join(dataPath,experimentID,f"{experimentID}.nwb")

session_description = f"test_{experimentID}_test"
experiment_description = "Pure tone in low and high DRC contrast with pupillometry"
keywords = ["2P", "CGC", "pupillometry", "DRC"]

lib.nwbScanImage.genNWBfromScanImage_pc(
    experimentID=experimentID, 
    dataPath=dataPath, 
    NWBoutputPath=outputNWBpath,
    session_description=session_description,
    experiment_description=experiment_description,
    keywords=keywords,
    **lib.nwbScanImage.PARAMS_nwbFilePC,
    **lib.nwbScanImage.PARAMS_imagingPC
)

In [ ]:
# after fix to file by file
import sys
sys.path.insert(1, '../')
import os
import lib.nwbScanImage

import importlib
importlib.reload(lib.nwbScanImage)

experimentID = 'AA0304'
# /media/DATA/backups/sutter2P_backup/D_drive/ZnT3_pupil/BIN2/AA0323
#  /media/DATA/backups/sutter2P_backup/D_drive/dataSince20210304/AA0308
dataPath = "/media/DATA/backups/sutter2P_backup/D_drive/dataSince20210304"
outputNWBpath = os.path.join(dataPath,experimentID,f"{experimentID}.nwb")

session_description = f"pupilPTinContrast_{experimentID}_C57BL6J"
experiment_description = "Pure tone in low and high DRC contrast with pupillometry"
keywords = ["2P", "CGC", "pupillometry", "DRC"]

lib.nwbScanImage.genNWBfromScanImage_pc(
    experimentID=experimentID, 
    dataPath=dataPath, 
    NWBoutputPath=outputNWBpath,
    session_description=session_description,
    experiment_description=experiment_description,
    keywords=keywords,
    **lib.nwbScanImage.PARAMS_nwbFilePC,
    **lib.nwbScanImage.PARAMS_imagingPC
)

## run NWB build over multiple directories

In [1]:
import os
import lib.nwbScanImage

import importlib
importlib.reload(lib.nwbScanImage)

dataPath = "/media/DATA/backups/sutter2P_backup/D_drive/dataSince20210304"

experiments = [
                'AA0304',
                'AA0308',
                'AA0310',
                'AA0312',
                'AA0314',
                'AA0315',
                'AA0318',
                'AA0350',
                'AA0353'
               ]

for experimentID in experiments:
    outputNWBpath = os.path.join(dataPath,experimentID,f"{experimentID}.nwb")

    if os.path.exists(outputNWBpath):
        print(f"NWB file already created for {experimentID}")
    else:
        session_description = f"pupilPTinContrast_{experimentID}_C57BL6J"
        experiment_description = "Pure tone in low and high DRC contrast with pupillometry"
        keywords = ["2P", "CGC", "pupillometry", "DRC"]

        lib.nwbScanImage.genNWBfromScanImage_pc(
            experimentID=experimentID, 
            dataPath=dataPath, 
            NWBoutputPath=outputNWBpath,
            session_description=session_description,
            experiment_description=experiment_description,
            keywords=keywords,
            **lib.nwbScanImage.PARAMS_nwbFilePC,
            **lib.nwbScanImage.PARAMS_imagingPC
            )

NWB file already created for AA0304
NWB file already created for AA0308
NWB file already created for AA0310
NWB file already created for AA0312
NWB file already created for AA0314
[('AA0315AAAA_00010_00001.tif', 170.0, 'preZX1'), ('AA0315AAAA_00011_00001.tif', 170.0, 'preZX1'), ('AA0315AAAA_00012_00001.tif', 170.0, 'preZX1'), ('AA0315AAAA_00013_00001.tif', 170.0, 'preZX1'), ('AA0315AAAA_00014_00001.tif', 170.0, 'preZX1'), ('AA0315AAAA_00015_00001.tif', 170.0, 'preZX1'), ('AA0315AAAA_00016_00001.tif', 170.0, 'preZX1'), ('AA0315AAAA_00017_00001.tif', 170.0, 'preZX1'), ('AA0315AAAA_00018_00001.tif', 170.0, 'preZX1'), ('AA0315AAAA_00019_00001.tif', 170.0, 'preZX1'), ('AA0315AAAA_00020_00001.tif', 170.0, 'preZX1'), ('AA0315AAAA_00021_00001.tif', 170.0, 'preZX1'), ('AA0315AAAA_00022_00001.tif', 170.0, 'preZX1'), ('AA0315AAAA_00023_00001.tif', 170.0, 'preZX1'), ('AA0315AAAA_00024_00001.tif', 170.0, 'preZX1'), ('AA0315AAAA_00025_00001.tif', 170.0, 'preZX1'), ('AA0315AAAA_00027_00001.tif', 120.

/home/pac/Documents/Python/nwb/venvNWB/lib/python3.12/site-packages/pynwb/file.py:472: UserWarning: Date is missing timezone information. Updating to local timezone.
  args_to_set['session_start_time'] = _add_missing_timezone(session_start_time)


added 2P data
added motion correction data
added ROI segmentation data


/home/pac/Documents/Python/nwb/venvNWB/lib/python3.12/site-packages/hdmf/container.py:536: UserWarning: The linked table for DynamicTableRegion 'rois' does not share an ancestor with the DynamicTableRegion.
  child._validate_on_set_parent()


added fluorescence trace data for ROIs
added stim table data
found pupillometry data for AA0315
added pupil radius data
added pupil video data
NWB write success to: /media/DATA/backups/sutter2P_backup/D_drive/dataSince20210304/AA0315/AA0315.nwb
[('AA0318AAAA_00011_00001.tif', 170.0, 'preZX1'), ('AA0318AAAA_00012_00001.tif', 170.0, 'preZX1'), ('AA0318AAAA_00013_00001.tif', 170.0, 'preZX1'), ('AA0318AAAA_00014_00001.tif', 170.0, 'preZX1'), ('AA0318AAAA_00015_00001.tif', 170.0, 'preZX1'), ('AA0318AAAA_00016_00001.tif', 170.0, 'preZX1'), ('AA0318AAAA_00017_00001.tif', 170.0, 'preZX1'), ('AA0318AAAA_00018_00001.tif', 170.0, 'preZX1'), ('AA0318AAAA_00019_00001.tif', 170.0, 'preZX1'), ('AA0318AAAA_00020_00001.tif', 170.0, 'preZX1'), ('AA0318AAAA_00021_00001.tif', 170.0, 'preZX1'), ('AA0318AAAA_00022_00001.tif', 170.0, 'preZX1'), ('AA0318AAAA_00023_00001.tif', 170.0, 'preZX1'), ('AA0318AAAA_00024_00001.tif', 170.0, 'preZX1'), ('AA0318AAAA_00025_00001.tif', 170.0, 'preZX1'), ('AA0318AAAA_00026_

In [ ]:
import os
import lib.nwbScanImage

import importlib
importlib.reload(lib.nwbScanImage)

dataPath = "/media/DATA/backups/sutter2P_backup/D_drive/dataSince20210304"

experiments = [
                'AA0304',
                'AA0308',
                'AA0310',
                'AA0312',
                'AA0314',
                'AA0315',
                'AA0318',
                'AA0350',
                'AA0353'
               ]

for experimentID in experiments:
    outputNWBpath = os.path.join(dataPath,experimentID,f"{experimentID}.nwb")

    if os.path.exists(outputNWBpath):
        print(f"NWB file already created for {experimentID}")
    else:
        session_description = f"pupilPTinContrast_{experimentID}_C57BL6J"
        experiment_description = "Pure tone in low and high DRC contrast with pupillometry"
        keywords = ["2P", "CGC", "pupillometry", "DRC"]

        lib.nwbScanImage.genNWBfromScanImage_pc(
            experimentID=experimentID, 
            dataPath=dataPath, 
            NWBoutputPath=outputNWBpath,
            session_description=session_description,
            experiment_description=experiment_description,
            keywords=keywords,
            **lib.nwbScanImage.PARAMS_nwbFilePC,
            **lib.nwbScanImage.PARAMS_imagingPC
            )

# issue AA0312 has moCorrROI_all but two treatment conditions (ROIs stayed the same), how to handle?